# Recusion Cellular Image Clasification with AI-Platform

# How to bring your model to AI Platform
Getting your model ready for predictions can be done in 5 steps:
1. Save your model to a file
2. Upload the saved model to [Google Cloud Storage](https://cloud.google.com/storage)
3. Create a model resource on AI Platform
4. Create a model version 
5. Make an online prediction

# Prerequisites
Before we begin, let’s cover some of the different tools you’ll use to get online prediction up and running on AI Platform. 

[Google Cloud Platform](https://cloud.google.com/) (GCP) lets you build and host applications and websites, store data, and analyze data on Google's scalable infrastructure.

[AI Platform](https://cloud.google.com/ml-engine/) is a managed service that enables you to easily build machine learning models that work on any type of data, of any size.

[Google Cloud Storage](https://cloud.google.com/storage/) (GCS) is a unified object storage for developers and enterprises, from live data serving to data analytics/ML to data archiving.

[Cloud SDK](https://cloud.google.com/sdk/) is a command line tool which allows you to interact with Google Cloud products. In order to run this notebook, make sure that Cloud SDK is [installed](https://cloud.google.com/sdk/downloads) in the same environment as your Jupyter kernel.


In [1]:
%env PROJECT_ID=mwpmltr
%env BUCKET_ID=cellular_img_demo
%env PACKAGE_PATH=trainer
%env MODULE_NAME=trainer.task
%env RUNTIME_VERSION=1.15
%env PYTHON_VERSION=3.7
%env REGION=us-central1

env: PROJECT_ID=mwpmltr
env: BUCKET_ID=cellular_img_demo
env: PACKAGE_PATH=trainer
env: MODULE_NAME=trainer.task
env: RUNTIME_VERSION=1.15
env: PYTHON_VERSION=3.7
env: REGION=us-central1


In [2]:
# Create BUCKET_NAME if it does not exist.
#!gsutil mb gs://${BUCKET_ID}

In [3]:
#copy data to the bucket

#This might take a while as we are copying images
#!gsutil -m cp -r gs://cellular-model-bucket/data gs://${BUCKET_ID}/data

In [ ]:
now = !date +"%Y%m%d_%H%M%S"
%env JOB_NAME=tensorflow_job_$now.s

!gcloud ai-platform jobs submit training $JOB_NAME \
  --package-path trainer/ \
  --module-name trainer.trainer \
  --region $REGION \
  --python-version 3.7 \
  --runtime-version 2.1 \
  --job-dir gs://${BUCKET_ID}/keras-job-dir-${JOB_NAME} \
  -- \
  --project-id $PROJECT_ID \
  --bucket-name ${BUCKET_ID} 
                
# Stream logs so that training is done before subsequent cells are run.
# Remove  '> /dev/null' to see step-by-step output of the model build steps.
!gcloud ai-platform jobs stream-logs $JOB_NAME > /dev/null

# Model should exit with status "SUCCEEDED"
!gcloud ai-platform jobs describe $JOB_NAME --format="value(state)"

env: JOB_NAME=tensorflow_job_20200903_065928
Job [tensorflow_job_20200903_065928] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe tensorflow_job_20200903_065928

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs tensorflow_job_20200903_065928
jobId: tensorflow_job_20200903_065928
state: QUEUED


In [ ]:
now = !date +"%Y%m%d_%H%M%S"
%env JOB_NAME=tensorflow_job_$now.s

!gcloud ai-platform jobs submit training $JOB_NAME \
  --scale-tier BASIC_TPU \
  --package-path trainer/ \
  --module-name trainer.trainer \
  --region $REGION \
  --python-version 3.7 \
  --runtime-version 2.1 \
  --job-dir gs://${BUCKET_ID}/keras-job-dir-${JOB_NAME} \
  -- \
  --project-id $PROJECT_ID \
  --bucket-name ${BUCKET_ID} \
  --tputrain True
                
# Stream logs so that training is done before subsequent cells are run.
# Remove  '> /dev/null' to see step-by-step output of the model build steps.
!gcloud ai-platform jobs stream-logs $JOB_NAME > /dev/null

# Model should exit with status "SUCCEEDED"
!gcloud ai-platform jobs describe $JOB_NAME --format="value(state)"